In [31]:
import textstat


In [46]:
import re
import math

class readability:
    
    def __init__(self):
        self.__rm_apostrophe = True
        self.__round_outputs = True
        self.__round_points = None
    
    def char_count(self, text: str, ignore_spaces: bool = True) -> int:
        """Count the number of characters in a text.

        Parameters
        ----------
        text : str
            A text string.
        ignore_spaces : bool, optional
            Ignore whitespaces if True. The default is True.

        Returns
        -------
        int
            Number of characters.

        """
        if ignore_spaces:
            text = re.sub(r"\s", "", text)
        return len(text)
    
    def lexicon_count(self, text: str, removepunct: bool = True) -> int:
        """Count types (words) in a text.

        If `removepunct` is set to True and
        the instance attribute `__rm_apostrophe` is set to False,
        English contractions (e.g. "aren't") are counted as one word.
        Hyphenated words are counted as a single word
        (e.g. "singer-songwriter").

        Parameters
        ----------
        text : str
            A text string.
        removepunct : bool, optional
            DESCRIPTION. The default is True.

        Returns
        -------
        count : int
            DESCRIPTION.

        """
        if removepunct:
            text = self.remove_punctuation(text)
        count = len(text.split())
        return count
    
    def sentence_count(self, text: str) -> int:
        """Count the sentences of the text.

        Parameters
        ----------
        text : str
            A text string.

        Returns
        -------
        int
            Number of sentences in `text`.

        """
        ignore_count = 0
        sentences = re.findall(r'\b[^.!?]+[.!?]*', text, re.UNICODE)
        for sentence in sentences:
            if self.lexicon_count(sentence) <= 2:
                ignore_count += 1
        return max(1, len(sentences) - ignore_count)
    
    def remove_punctuation(self, text: str) -> str:
        """Remove punctuation.

        If the instance attribute `__rm_apostrophe` is set to True, all
        punctuation is removed, including apostrophes.
        If the instance attribute `__rm_apostrophe` is set to False,
        punctuation is removed with the exception of apostrophes in common
        English contractions.
        Hyphens are always removed.

        Parameters
        ----------
        text : str
            A text string.

        Returns
        -------
        text : TYPE
            DESCRIPTION.

        """
        if self.__rm_apostrophe:
            # remove all punctuation
            punctuation_regex = r"[^\w\s]"
        else:
            # replace single quotation marks with double quotation marks but
            # keep apostrophes in contractions
            text = re.sub(r"\'(?![tsd]\b|ve\b|ll\b|re\b)", '"', text)
            # remove all punctuation except apostrophes
            punctuation_regex = r"[^\w\s\']"

        text = re.sub(punctuation_regex, '', text)
        return text
    
    def _legacy_round(self, number: float, points: int = 0) -> float:
        """Round `number`, unless the attribute `__round_outputs` is `False`.

        Round floating point outputs for backwards compatibility.
        Rounding can be turned off by setting the instance attribute
        `__round_outputs` to False.

        Parameters
        ----------
        number : float
        points : int, optional
            The number of decimal digits to return. If the instance attribute
            `__round_points` is not None, the value of `__round_point` will
            override the value passed for `points`. The default is 0.

        Returns
        -------
        float

        """
        points = self.__round_points if (
            self.__round_points is not None) else points
        if self.__round_outputs:
            p = 10 ** points
            return float(
                math.floor((number * p) + math.copysign(0.5, number))) / p
        else:
            return number
    
    def automated_readability_index(self, text: str) -> float:
            r"""Calculate the Automated Readability Index (ARI).

            Parameters
            ----------
            text : str
                A text string.

            Returns
            -------
            float
                The ARI for `text`.

            Notes
            -----
            The ARI is calculated as:

            .. math::

                (4.71*n\ characters/n\ words)+(0.5*n\ words/n\ sentences)-21.43

            """
            chrs = self.char_count(text)
            words = self.lexicon_count(text)
            sentences = self.sentence_count(text)
            try:
                a = float(chrs) / float(words)
                b = float(words) / float(sentences)
                readability = (
                        (-0.10073196 * self._legacy_round(a, 2))
                        + (0.51377167 * self._legacy_round(b, 2))
                        - 0.20376572645061142)
                return self._legacy_round(readability, 1)
            except ZeroDivisionError:
                return 0.0

In [7]:
import glob

c1 = []
c2 = []

for i in glob.glob('story/*.txt'):
    with open(i) as f:
        lines = f.read()
        lines = re.sub(r"\n", '', lines)
        c1.append(lines.split('----------')[0])
        c2.append(lines.split('----------')[1])        
        

In [45]:
from sklearn.linear_model import LinearRegression
import numpy as np


def train_air_params(c1, c2):
    r = readability()
    y = []
    x = []
    model = LinearRegression()
    for i, j in zip(c1, c2):
        y.append(textstat.automated_readability_index(i))
        chrs = r.char_count(j)
        words = r.lexicon_count(j)
        sentences = r.sentence_count(j)
        try:
            a = float(chrs) / float(words)
            b = float(words) / float(sentences)
            a = r._legacy_round(a, 2)
            b = r._legacy_round(b, 2)
            x.append([a, b])
        except ZeroDivisionError:
            return 0.0
    model.fit(x, y)
    return model.coef_, model.intercept_

train_air_params(c1, c2)

(array([-0.10073196,  0.51377167]), -0.20376572645061142)

In [48]:
r = readability()
r.automated_readability_index('رگرسیون خطی یا تنازل خطی یا وایازی خطی[الف] یکی از روش‌های تحلیل رگرسیون است. رگرسیون یک نوع مدل آماری‌ست برای پیش‌بینی یک متغیر از روی یک یا چند متغیر دیگر. رگرسیون خطی نوعی تابع پیش‌بینی‌کننده خطی است که در آن متغیر وابسته — متغیری که قرار است پیش‌بینی شود — به صورت ترکیبی خطی از متغیرهای مستقل پیش‌بینی می‌شود، بدین معنی که هر کدام از متغیرهای مستقل در ضریبی که در فرایند تخمین برای آن متغیر به‌دست آمده ضرب می‌شود؛ جواب نهائی مجموع حاصل‌ضرب‌ها به علاوه یک مقدار ثابت خواهد بود که آن هم در فرایند تخمین به‌دست آمده‌است. ساده‌ترین نوع رگرسیون خطی، رگرسیون خطی ساده است که بر خلاف رگرسیون خطی چندگانه، تنها یک متغیر مستقل دارد. نوع دیگر رگرسیون خطی رگرسیون خطی چندمتغیره است که در آن به جای پیش‌بینی یک متغیر وابسته چندین متغیر وابسته پیش‌بینی می‌شود.')


13.5

In [49]:
r.automated_readability_index('روزی روزگاری مرغی در یک مزرعه زنگی میکرد که بخاطر پرهای قرمزش همه او را پر قرمزی صدا میکردند. روزی پر قرمزی در مزرعه در حال گشت و گذار و دانه خوردن بود که روباهی او را دید و آب از دهانش به راه افتاد. سریع به خانه رفت و به همسرش گفت قابلمه را پر از آب کند و روی گاز بگذارد تا او ناهار را بیاورد. بعد دوباره به مزرعه برگشت. وقتی پرقرمزی اصلا حواسش نبود. پیش از آنکه بتواند کمک بخواهد، او را گرفت و در یک گونی انداخت. و بعد خوشحال راه افتاد به سمت خانه. دوست پرقرمزی که یک کبوتر بود، همه ی داستان را تماشا میکرد و برای نجات دوستش سریع یک نقشه کشید. کبوتر رفت و سر راه روباه نشست و وانمود کرد که پایش شکسته است. روباه تا او را دید خیلی خوشحال شد و با خودش فکر کرد امروز ناهار مفصلی میخورد. گونی را روی زمین گذاشت و به سمت کبوتر رفت تا او را بگیرد. کبوتر هم آرام آرام عقب میرفت. پرقرمزی تا دید که روباه حواسش به کبوتر است از توی گونی بیرون آمد، یک سنگ داخل گونی گذاشت و فرار کرد. کبوتر وقتی دید دوستش به اندازه کافی دور شده، شروع به پرواز کرد و بالای درختی نشست. روباه هم که ناامید شده بود به سمت گونی رفت و آن را برداشت و به خانه رفت.')


7.3